In [2]:
import pandas as pd 
import matplotlib.pyplot as plt

In [9]:
df = pd.read_excel("./data/studyGroup -original.xlsx")

In [15]:
df.head()

,ID,Gender,Age,Employment,Marital,children,MinorChildren,AcademicalLevel,DiabetesType,DurationDiabetes,...,Energy27,BlueDownhearted28,Wornout29,happy30,Tired31,P32hysicalEmotionOnSocial2,SickEasier33,AsEqual34,worsen35,ExcellentHealth36
0,1,male,47,other,married,yes,5,secondary/high school,type2,12,...,Most of the time,None of the time,Some of the time,All of the time,A good bit of the time,Some of the time,Don’t know,Mostly false,Mostly false,Mostly true
1,2,male,64,no,married,yes,1,college,type2,26,...,All of the time,None of the time,Some of the time,All of the time,A good bit of the time,Some of the time,Definitely false,Mostly true,Definitely false,Definitely true
2,3,male,67,no,widowed,yes,0,college,type2,15,...,All of the time,Some of the time,Some of the time,All of the time,Some of the time,None,Don’t know,Mostly true,Mostly false,Definitely true
3,4,male,41,other,married,yes,4,khalwa,type2,7,...,A good bit of the time,A little of the time,A good bit of the time,All of the time,A good bit of the time,A little of the time,Don’t know,Mostly true,Mostly false,Definitely true
4,5,male,57,no,married,yes,0,secondary/high school,type2,17,...,A good bit of the time,None of the time,Some of the time,All of the time,A good bit of the time,Most of the time,Don’t know,Mostly true,Don’t know,Mostly true


In [11]:
print(df.describe())

              ID        Age  MinorChildren  DurationDiabetes  DurationUlcer
count  60.000000  60.000000      60.000000         60.000000      60.000000
mean   30.500000  52.833333       1.833333         12.616667      73.450000
std    17.464249  13.971116       2.394674         12.838094     117.360725
min     1.000000  23.000000       0.000000          1.000000       3.000000
25%    15.750000  42.750000       0.000000          4.750000      10.000000
50%    30.500000  54.000000       1.000000         10.000000      18.500000
75%    45.250000  65.000000       3.000000         16.250000      90.000000
max    60.000000  81.000000      10.000000         90.000000     515.000000


In [13]:
print(df.dtypes)

ID                               int64
Gender                          object
Age                              int64
Employment                      object
Marital                         object
children                        object
MinorChildren                    int64
AcademicalLevel                 object
DiabetesType                    object
DurationDiabetes                 int64
MedicineType                    object
Adherence                       object
DurationUlcer                    int64
Smoking                         object
PainfulUlcer                    object
FormerUlcers                    object
FormerAmputations               object
I1nGeneral1                     object
C2ompareToOneYear               object
L3imitVigorousActivityNow       object
L4imitModerateActivityNow       object
c5arryingGroceriesNow           object
s6everalFlightsNow              object
o7neFlightNow                   object
B8endingKneelingStoopingNow     object
m9oreThanAmileNow        

In [14]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

,Gender,Employment,Marital,children,AcademicalLevel,DiabetesType,MedicineType,Adherence,Smoking,PainfulUlcer,...,Energy27,BlueDownhearted28,Wornout29,happy30,Tired31,P32hysicalEmotionOnSocial2,SickEasier33,AsEqual34,worsen35,ExcellentHealth36
0,male,other,married,yes,secondary/high school,type2,insulin,yes,no,no,...,Most of the time,None of the time,Some of the time,All of the time,A good bit of the time,Some of the time,Don’t know,Mostly false,Mostly false,Mostly true
1,male,no,married,yes,college,type2,insulin,yes,no,no,...,All of the time,None of the time,Some of the time,All of the time,A good bit of the time,Some of the time,Definitely false,Mostly true,Definitely false,Definitely true
2,male,no,widowed,yes,college,type2,insulin,yes,no,no,...,All of the time,Some of the time,Some of the time,All of the time,Some of the time,None,Don’t know,Mostly true,Mostly false,Definitely true
3,male,other,married,yes,khalwa,type2,insulin,yes,yes,no,...,A good bit of the time,A little of the time,A good bit of the time,All of the time,A good bit of the time,A little of the time,Don’t know,Mostly true,Mostly false,Definitely true
4,male,no,married,yes,secondary/high school,type2,insulin,yes,no,no,...,A good bit of the time,None of the time,Some of the time,All of the time,A good bit of the time,Most of the time,Don’t know,Mostly true,Don’t know,Mostly true


In [16]:
obj_df["Gender"].value_counts()

male      36
female    24
Name: Gender, dtype: int64

In [17]:
obj_df.describe()

,Gender,Employment,Marital,children,AcademicalLevel,DiabetesType,MedicineType,Adherence,Smoking,PainfulUlcer,...,Energy27,BlueDownhearted28,Wornout29,happy30,Tired31,P32hysicalEmotionOnSocial2,SickEasier33,AsEqual34,worsen35,ExcellentHealth36
count,60,60,60,60,60,60,60,60,60,60,...,60,60,60,60,60,60,60,60,60,60
unique,2,3,4,2,5,2,2,2,2,2,...,6,3,5,5,5,5,5,5,5,5
top,male,no,married,yes,college,type2,insulin,yes,no,yes,...,A good bit of the time,None of the time,Some of the time,All of the time,A good bit of the time,A little of the time,Don’t know,Don’t know,Mostly false,Mostly true
freq,36,33,48,52,17,53,51,56,53,33,...,17,28,22,18,27,15,31,25,23,25
